# Step 3: Calculate OD

In [1]:
import os, sys, time, importlib
import osmnx

import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
sys.path.append("../../../GOSTnets")
import GOSTnets as gn

# pip install osmium
# import osmium, logging
# import shapely.wkb as wkblib

from shapely.geometry import LineString, Point

In [2]:
# This is a Jupyter Notebook extension which reloads all of the modules whenever you run the code
# This is optional but good if you are modifying and testing source code
%load_ext autoreload
%autoreload 2

In [3]:
from GOSTnets.load_traffic2 import *

In [4]:
#read_nodes = pd.read_csv('./vavuniya_unclean_nodes.csv')

In [5]:
#read_nodes[:5]

In [6]:
# read graph
G = nx.read_gpickle('./sri_lanka_unclean2_w_time_largest_20200616.pickle')

In [7]:
len(G.edges)

1675316

In [8]:
gn.example_edge(G, 5)

(1650104033, 6236632580, {'osm_id': 4860427, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fe3b70f5490>, 'infra_type': 'trunk', 'min_speed': 23.0, 'max_speed': 37.0, 'mean_speed': 26.793650793650794, 'length': 0.009757789924575978, 'speed': 26.793650793650794, 'time': 1.3110585040840237, 'mode': 'drive'})
(1650104033, 970058024, {'osm_id': 152177791, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fe3b70f5750>, 'infra_type': 'trunk', 'min_speed': 23.0, 'max_speed': 37.0, 'mean_speed': 26.793650793650794, 'length': 0.0160020254611234, 'speed': 26.793650793650794, 'time': 2.150035174515869, 'mode': 'drive'})
(6236632580, 6236632579, {'osm_id': 4860427, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fe3b70f5810>, 'infra_type': 'trunk', 'min_speed': 23.0, 'max_speed': 37.0, 'mean_speed': 26.793650793650794, 'length': 0.005235256075996137, 'speed': 26.793650793650794, 'time': 0.7034099988364477, 'mode': 'drive'})
(6236632580, 1650104033,

## load origins

In [9]:
origins = gpd.read_file('./origins_destinations/cities_top10.shp')

In [10]:
origins

,osm_id,code,fclass,population,name,country,geometry
0,44240758,1001,city,27500,Nuwara Eliya,LKA,POINT (80.76707 6.97382)
1,50794342,1005,national_capital,753000,Colombo,LKA,POINT (79.85385 6.93500)
2,258049423,1001,city,40800,Matale,LKA,POINT (80.62342 7.47213)
3,330173100,1001,city,45400,Puttalam,LKA,POINT (79.82866 8.03019)
4,335253618,1001,city,99100,Trincomalee,LKA,POINT (81.23450 8.57643)
5,2050947471,1001,city,600000,Jaffna,LKA,POINT (80.00930 9.66509)
6,2908772560,1001,city,125000,Kandy,LKA,POINT (80.63508 7.29309)
7,3289447451,1001,city,20000,Gampaha,LKA,POINT (79.99514 7.09256)
8,3639539242,1001,city,110000,Matara,LKA,POINT (80.54829 5.94782)
9,3639546887,1001,city,120000,Galle,LKA,POINT (80.21495 6.03281)


## load destinations

In [11]:
destinations = gpd.read_file('./origins_destinations/tourism_on_land_wgs84.shp')

In [12]:
destinations

,field_1,Loc_name,LongLabel,ShortLabel,Addr_type,Type,PlaceName,Place_addr,StAddr,Block,...,NEAR_DIST,NEAR_X,NEAR_Y,nearCtyPop,name,NN,NN_dist,x,y,geometry
0,0,World,"Embekka Katharagama Dewalaya, Embekka, LKA",Embekka Katharagama Dewalaya,POI,Temple,Embekka Katharagama Dewalaya,Embekka,None,None,...,11110.595704,459719.265399,806165.680488,125000,Kandy,1.743127e+09,14.430923,452313.8385,797882.8676,POINT (80.56806 7.21811)
1,1,None,None,None,None,None,None,None,None,None,...,15374.643812,458448.971316,825958.949979,40800,Matale,3.645434e+09,120.831359,471631.7764,818047.3558,POINT (80.74294 7.40064)
2,2,World,"Royal Palace Park-Kandy, Keerthie Sri Rajasing...",Royal Palace Park-Kandy,POI,Park,Royal Palace Park-Kandy,"Keerthie Sri Rajasinghe Mawatha, Kandy",Keerthie Sri Rajasinghe Mawatha,None,...,482.167726,459719.265399,806165.680488,125000,Kandy,3.804267e+09,7.492919,459987.5274,805765.0295,POINT (80.63751 7.28947)
3,3,World,"Kandy, LKA",Kandy,Locality,City,Kandy,None,None,None,...,446.690670,459719.265399,806165.680488,125000,Kandy,5.507322e+09,29.207214,460114.9554,806372.9532,POINT (80.63866 7.29497)
4,4,World,"Bauddha Bala Mandalaya, Akkara Road, Ranawana ...",Bauddha Bala Mandalaya,POI,Temple,Bauddha Bala Mandalaya,"Akkara Road, Ranawana East",Akkara Road,None,...,4130.237703,459719.265399,806165.680488,125000,Kandy,2.342916e+09,110.833504,458356.3441,810064.5661,POINT (80.62270 7.32835)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,409,World,"Wasana Trade Point, Temple Road, Negombo, LKA",Wasana Trade Point,POI,Business Facility,Wasana Trade Point,"Temple Road, Negombo",Temple Road,None,...,18946.453259,389028.597467,784101.180029,20000,Gampaha,3.804238e+09,24.739240,374037.4230,795687.0671,POINT (79.85915 7.19703)
376,410,World,"Post Office-Wadduwa, Wadduwa Station Road, Wad...",Post Office-Wadduwa,POI,Post Office,Post Office-Wadduwa,"Wadduwa Station Road, Wadduwa South",Wadduwa Station Road,None,...,31162.749630,373380.454933,766715.939941,753000,Colombo,5.397261e+09,0.948475,381667.2222,736675.1934,POINT (79.92945 6.66347)
377,411,World,"Madusanka, A8, Wekada West, LKA",Madusanka,POI,Grocery,Madusanka,"A8, Wekada West",A8,None,...,25714.185503,373380.454933,766715.939941,753000,Colombo,1.446610e+09,4.806681,379876.6902,741835.8623,POINT (79.91315 6.71011)
378,412,World,"Post Office-Beruwala, Mangala Road, Paranakade...",Post Office-Beruwala,POI,Post Office,Post Office-Beruwala,"Mangala Road, Paranakade",Mangala Road,None,...,53062.770683,373380.454933,766715.939941,753000,Colombo,1.606697e+09,91.068186,387596.1659,715592.8415,POINT (79.98347 6.47290)


In [13]:
origins_gdf = gn.pandana_snap_c(G, origins, source_crs = 'epsg:4326', target_crs = 'epsg:32644')

In [14]:
#origins_gdf

In [15]:
origins_list = list(set(origins_gdf.NN))

In [16]:
destinations_gdf = gn.pandana_snap_c(G, destinations, source_crs = 'epsg:4326', target_crs = 'epsg:32644')

In [17]:
#destinations_gdf

In [18]:
destinations_list = list(set(destinations_gdf.NN))

In [19]:
#destinations_list

## Calculate OD

In [20]:
# It will use the default weight of 'time'
import time

start = time.time()

OD = gn.calculate_OD(G, origins_list, destinations_list, fail_value = 9999999)

end = time.time()
print(end - start)

59.04262328147888


In [21]:
#OD

In [22]:
OD_df = pd.DataFrame(OD, index = origins_list, columns = destinations_list)

In [23]:
OD_df

,2291208197,1667080206,6991570964,6117715989,1743126554,2100457519,1185558575,130445367,2085208123,4047450172,...,4169301972,3772792789,3772792790,1730404311,258523099,6564888541,3216934896,1506164725,1657419767,4621961215
4140985760,18510.074391,12507.295948,10194.326855,17017.046783,11058.179415,14588.843982,11616.506298,14153.017294,21992.550175,9847.193878,...,21572.482236,10785.029235,10774.574424,20191.061674,11498.594777,12598.994605,12013.650506,15059.136708,4829.209158,10138.057196
4784044133,18168.790483,16420.960320,15970.771248,246.884367,8132.610523,26436.198542,16370.683271,18429.611369,10032.738931,7421.909044,...,19549.813793,16542.592663,16551.018817,12617.043637,15568.322303,24446.349165,9460.677768,17173.575547,17919.415079,15914.501589
1734550181,22253.893418,5577.871197,7919.887895,15703.435161,15053.684958,28808.955754,6628.789813,4084.405129,12451.921281,15797.568395,...,6909.207781,7155.992360,7166.447171,5608.141428,20767.059336,26819.106377,11771.937683,2828.369308,19234.270845,8107.092454
4004620073,15357.120897,12269.390354,10880.557578,7577.372332,2363.735176,19278.819754,11542.389592,13915.111700,16917.251506,175.355078,...,21334.576642,11471.259958,11460.805147,17386.911484,9107.722890,17288.970377,7586.305618,14821.231114,11054.161876,10824.287919
3379679501,23632.568454,4604.393388,6946.410085,17082.110198,14080.207149,27835.477944,5655.312004,2641.929211,13830.596317,14824.090585,...,9037.610689,6182.514551,6192.969362,6986.816464,19793.581527,25845.628567,10798.459874,248.574822,18260.793036,7133.614644
3289453904,20834.449315,4373.582498,2984.749722,13897.581977,8109.595324,21754.640756,3646.581736,6019.303844,14196.753964,8853.478760,...,13438.768786,3575.452102,3564.997291,12057.348224,13822.969702,19764.791379,5474.782889,6925.423258,12179.955848,2928.480063
1243386867,10050.216305,21028.939802,19640.107026,21176.982762,15603.274375,7627.850486,20301.939039,22674.661147,25899.034561,13597.703797,...,30094.126090,20230.809405,20220.354595,24473.158962,6771.671943,10369.806621,17918.607278,23580.780562,7247.822892,19583.837366
7098642100,23877.890534,28003.185631,26374.805251,29229.796481,23656.088094,9325.359153,27276.184869,29648.906977,34941.923384,21650.517516,...,37068.371919,26965.507630,26955.052820,33516.047784,17718.037277,5435.996178,25167.001175,30555.026391,12465.607681,26318.535591
1727494683,13600.798789,13830.619341,12441.786565,9922.327601,4401.428912,16760.416039,13103.618578,15476.340687,19262.206776,2343.048636,...,22895.805629,13032.488944,13022.034134,18380.281612,6589.319175,14770.566663,9819.827132,16382.460101,8611.520671,12385.516906
5385270047,22760.369817,4335.427941,527.037160,15823.502479,10035.515826,23551.239017,2070.903425,6179.747102,15114.659932,10779.399262,...,13599.212044,1389.961431,1379.506621,12217.791482,15748.890204,21561.389640,7361.264357,7085.866516,13976.554108,257.357572


In [24]:
#OD_df.min(axis=0)

### Now we need to find the nearest city for each tourist point, and based on this assign a group of tourist points to each of the 10 cities

In [25]:
# test
# first column is the destination (tourist point), and the second column is the nearest city
OD_df.idxmin(axis=0)

2291208197    1243386867
1667080206    5385270047
6991570964    5385270047
6117715989    4784044133
1743126554    4004620073
                 ...    
6564888541    7098642100
3216934896    3289453904
1506164725    3379679501
1657419767    4140985760
4621961215    5385270047
Length: 313, dtype: int64

In [26]:
#OD_df.idxmin(axis=0).to_csv('./nodes_and_associated_nearest_city.csv')

In [27]:
#OD_df.idxmin(axis=0).to_frame(0)

### takes the min index value of each column, then groups by city (first index (0)) and takes the first entry

In [28]:
groupby_obj = OD_df.idxmin(axis=0).to_frame(0).groupby(0)[0]

In [29]:
groupby_obj

In [30]:
# visualize groupby_obj
#groupby_obj.apply(list)

In [31]:
#type(groupby_obj.apply(list))

In [32]:
# a nice way to visualize the groupby_obj
#groupby_obj.describe()

### create a dictionary that associates assigned tourist points with each city

In [33]:
city_tourist_pt_dict = {}
for name, group in groupby_obj:
    #print(group)
    for items in group.iteritems(): 
        #print(items[1])
        if items[1] not in city_tourist_pt_dict:
            city_tourist_pt_dict[items[1]] = [items[0]]
        else:
            #append value to list in dict value
            city_tourist_pt_dict[items[1]].append(items[0])
    #print(type(group))
    #print(group.head(1))

    #print(name)
    #print(city_tourist_pt_dict[group])

In [34]:
city_tourist_pt_dict

{1243386867: [2291208197,
  2100457519,
  1242787912,
  1242787995,
  3806787750,
  4248172715,
  5580828849,
  3851290872,
  4579338537,
  1884656092,
  1880627748,
  6177931854,
  7076053840,
  3985464364,
  4376980578,
  3315891502,
  2278210866,
  3893372372,
  7025200613,
  2341660185,
  6021684779,
  1676588668,
  3350242971,
  2477387622,
  1775835028,
  3165939618,
  5965445027],
 1727494683: [4784988278,
  3208652929,
  6988599546,
  2833357077,
  6181355836,
  3213648201,
  2646372818,
  5441882639,
  5035668064,
  2051261052,
  2279244515,
  1692965634,
  2831541077,
  3748668362,
  3218396149,
  1241535503,
  4924478499,
  1241527387,
  5644231795,
  1241527444,
  258057398,
  3280149722,
  1241527517,
  2833347872,
  2821176699,
  5483879958,
  2297833003,
  3218405040,
  1241534200,
  2303446810,
  3999442747,
  2799732582,
  2799732584,
  3472453531,
  258523099],
 1734550181: [4893239380,
  3587975265,
  1745895527,
  1741201515,
  610939019,
  1722822882,
  2145161538,

## Loop through dictionary in order to do a calculate_OD for each city's nearest tourist sites

In [35]:
OD = {}
OD_df_dict = {}
for city,tourist_pts in city_tourist_pt_dict.items():
    OD[city] = gn.calculate_OD(G, [city], tourist_pts, fail_value = 9999999)
    OD_df_dict[city] = pd.DataFrame(OD[city], index = [city], columns = tourist_pts)

In [36]:
sample_df = OD_df_dict[4784044133]
sample_df

,6117715989,2085208123,2078574693,1146790064,5497589950,1709670615,7083442431,5596123398,3686420776,849219917,...,4342594760,3301905899,198553180,2408066657,3615147754,1713880855,5497589553,2057039735,2057039753,77682630
4784044133,246.884367,10032.738931,5795.916461,5595.063132,2950.074792,7434.484289,487.724105,108.423476,1637.208436,1904.107684,...,3726.12794,5532.948146,432.959124,1479.318821,8967.031034,5902.910295,2800.297799,5480.234495,5469.468574,25.618408


In [37]:
OD_df_dict

{1243386867:               2291208197   2100457519  1242787912  1242787995   3806787750  \
 1243386867  10050.216305  7627.850486  568.038849  232.102591  1622.532349   
 
             4248172715   5580828849   3851290872  4579338537    1884656092  \
 1243386867  164.500294  5897.514957  3067.214478  8049.56954  14326.260821   
 
             ...   3893372372  7025200613   2341660185   6021684779  \
 1243386867  ...  8689.623353  916.141028  2099.758945  5566.063076   
 
              1676588668   3350242971  2477387622  1775835028   3165939618  \
 1243386867  9953.941107  15283.23918  9902.33526  540.556013  6677.291317   
 
              5965445027  
 1243386867  6219.852061  
 
 [1 rows x 27 columns],
 1727494683:              4784988278  3208652929   6988599546   2833357077   6181355836  \
 1727494683  3914.615286  6070.48506  7779.166397  7680.141252  3649.504852   
 
              3213648201   2646372818   5441882639   5035668064   2051261052  \
 1727494683  3044.576043  7569.810

### time calculateOD time for one city and its set of tourist points

In [38]:
start = time.time()
OD[4784044133] = gn.calculate_OD(G, [4784044133], tourist_pts, fail_value = 9999999)
print(time.time() - start)

6.045063257217407


In [39]:
# sum of all shortest routes
OD[4784044133][0].sum()

336918.7513906677

## Now work on generating routes and visualizing them

In [40]:
#gn.example_edge(G,2)

In [41]:
#G.get_edge_data(4784044133, 3819995230)

In [42]:
#G.get_edge_data(4784044133, 3819995230)[0]['geometry'].coords

In [43]:
from shapely.ops import linemerge
from itertools import islice

In [44]:
#mapbox_traffic_mean_speeds = {
#    'secondary': 34,
#    'secondary_link': 9,
#    'tertiary': 25,
#    'tertiary_link': 13,
#    'residential': 20,
#    'unclassified': 20,
#}

In [45]:
maxspeed_mean_speeds = {
    'secondary': 50,
    'secondary_link': 45,
    'tertiary': 40,
    'tertiary_link': 40,
    'residential': 25,
    'unclassified': 25,
}

In [46]:
def tabulate_edges(route):
    edge_table = []
    route_geometry = LineString()
    for idx in range(0, len(route) - 1):
        #edge_table.append([route[idx], route[idx+1]])
        # look up line
        #print('to node')
        #print(route[idx])
        #print('from node')
        #print(route[idx+1])
        edge_geometry = G.get_edge_data(route[idx],route[idx+1])[0]['geometry']
        # get edge speed
        edge_speed = G.get_edge_data(route[idx],route[idx+1])[0]['speed']
        #print('print edge_speed')
        #print(edge_speed)
        
        # compare edge speed to median speed
        rural_roads_list = ['residential','secondary','secondary_link','tertiary','tertiary_link','unclassified']
        
        edge_infra_type = G.get_edge_data(route[idx],route[idx+1])[0]['infra_type']
        
        edge_length = G.get_edge_data(route[idx],route[idx+1])[0]['length']
        edge_time = G.get_edge_data(route[idx],route[idx+1])[0]['time']
        
        mean_speed = G.get_edge_data(route[idx],route[idx+1])[0]['mean_speed']
        
        try:
            edge_imp_cost = G.get_edge_data(route[idx],route[idx+1])[0]['imp_cost']
        except:
            edge_imp_cost = 0
            pass
        
        if mean_speed > 0:      
            if edge_infra_type in rural_roads_list:
                #print('print improved speed')
                #print(mapbox_traffic_mean_speeds[edge_infra_type])
                #assums that current edge lenght is in km
                new_time_s = (edge_length / maxspeed_mean_speeds[edge_infra_type]) * 3600
                #new_time_s = ((edge_length / 1000) / maxspeed_mean_speeds[edge_infra_type]) * 3600
                #print("print new_time_s")
                #print(new_time_s)
                #print('print old time')
                #print(edge_time)
                edge_savings = edge_time - new_time_s
                # assign savings time
                edge_table.append([route[idx], route[idx+1], edge_savings, edge_imp_cost, edge_length, edge_infra_type, mean_speed, edge_geometry])
                # data['sec_saved'] = data['length'] / new_time
                # data['improvement_cost'] = 174861 * data['length'] / 1000
            else:
                # data['sec_saved'] = 0
                # very high number
                # data['improvement_cost'] = 10000000000
                #print("not a rural road")
                edge_table.append([route[idx], route[idx+1], 0, 0, edge_length, edge_infra_type, mean_speed, edge_geometry])
        else:
            edge_table.append([route[idx], route[idx+1], 0, 0, edge_length, edge_infra_type, mean_speed, edge_geometry])
            
        route_geometry = route_geometry.union(edge_geometry)
        
    #print('print route_geometry')
    #print(route_geometry)
    
    return(edge_table, route_geometry)

In [47]:
def generate_complete_edges_and_routes(input_df):

    LIMIT = 1000000

    complete_edges = []
    complete_routes = []

    count = 0

    # for origin, row in sample_df.iterrows(): 
    for origin, row in islice(input_df.iterrows(), LIMIT):    
        for destination, value in islice(row.items(), LIMIT):
            try:
                origin = int(origin)
            except:
                pass
            try:
                destination = int(destination)
            except:
                pass

            count = count + 1

            route = nx.dijkstra_path(G, origin, destination, weight = 'time')
            #path_edges = zip(route,route[1:])
            #print('print path_edges')
            #print(list(path_edges))
            # print(route)
            edge_table, route_geometry = tabulate_edges(route)
            #print('print edge_table:')
            #print(edge_table)
            complete_edges = complete_edges + edge_table
            #print('route_time')
            #print(value)
            complete_routes.append([edge_table[0][0], edge_table[-1][1], value, route_geometry])
            #print('edge_table[:-1]')
            #print(edge_table[-1][1])
            
    # convert complete_edges to gdf
    complete = pd.DataFrame(complete_edges, columns = ['o', 'd', 'sec_saved', 'imp_cost', 'length', 'infra_type', 'mean_speed', 'geometry'])
    complete['w'] = 1
    complete_count = complete.groupby(['o','d']).agg(
        {
            'w':"count",
            'sec_saved': sum,
            'imp_cost': sum,
            'mean_speed': 'first',
            'length':'first',
            'infra_type':'first',
            'geometry':'first'
        }
    )
    complete_count.reset_index(inplace = True)
    complete_count['o'] = complete_count['o'].astype(str)
    complete_count['d'] = complete_count['d'].astype(str)
    complete_count['weighted_sec_saved'] = complete_count.w * complete_count.sec_saved
    complete_count.sort_values(by=['weighted_sec_saved'], ascending=False)
    complete_count_gdf = gpd.GeoDataFrame(complete_count, crs = 'epsg:4326')
    
    # convert complete_routes to gdf
    complete_routes_df = pd.DataFrame(complete_routes, columns = ['origin','destination','time','geometry'])
    complete_routes_gdf = gpd.GeoDataFrame(complete_routes_df, crs = 'epsg:4326')
        
    return [complete_count_gdf, complete_routes_gdf]

In [48]:
#results = generate_complete_edges_and_routes(OD_df[4784044133])

In [57]:
OD_df_dict

{1243386867:               2291208197   2100457519  1242787912  1242787995   3806787750  \
 1243386867  10050.216305  7627.850486  568.038849  232.102591  1622.532349   
 
             4248172715   5580828849   3851290872  4579338537    1884656092  \
 1243386867  164.500294  5897.514957  3067.214478  8049.56954  14326.260821   
 
             ...   3893372372  7025200613   2341660185   6021684779  \
 1243386867  ...  8689.623353  916.141028  2099.758945  5566.063076   
 
              1676588668   3350242971  2477387622  1775835028   3165939618  \
 1243386867  9953.941107  15283.23918  9902.33526  540.556013  6677.291317   
 
              5965445027  
 1243386867  6219.852061  
 
 [1 rows x 27 columns],
 1727494683:              4784988278  3208652929   6988599546   2833357077   6181355836  \
 1727494683  3914.615286  6070.48506  7779.166397  7680.141252  3649.504852   
 
              3213648201   2646372818   5441882639   5035668064   2051261052  \
 1727494683  3044.576043  7569.810

In [ ]:
# keys completed
# 1243386867
# 1727494683
# 1734550181
# 3289453904
# 3379679501
# 4004620073
# 4140985760
# 4784044133
# 5385270047
# 7098642100

In [76]:
import time
start = time.time()
results[7098642100] = generate_complete_edges_and_routes(OD_df_dict[7098642100])
print(time.time() - start)

86.73304104804993


In [77]:
key = 7098642100
file_name = f"./output_edges_shapefiles_20200616/weighted_sec_saved_edges_{key}.shp"
print(file_name)
results[key][0].to_file(driver = 'ESRI Shapefile', filename = file_name)

./output_edges_shapefiles_20200616/weighted_sec_saved_edges_7098642100.shp


In [56]:
import time

start = time.time()

results = {}

count = 0
for key in OD_df_dict:
    while count > 1 and count < 6:
        results[key] = generate_complete_edges_and_routes(OD_df_dict[key])
    count += 1
    
print(time.time() - start)

KeyboardInterrupt: 

In [ ]:
for key in results:
    # print edges
    #print(results[key][0])
    file_name = f"./output_edges_shapefiles_20200616/weighted_sec_saved_edges_{key}.shp"
    print(file_name)
    results[key][0].to_file(driver = 'ESRI Shapefile', filename = file_name)

In [51]:
results[1243386867][0]

,o,d,w,sec_saved,imp_cost,mean_speed,length,infra_type,geometry,weighted_sec_saved
0,258522955,4086541800,1,0.0,0.0,47.401786,0.025563,trunk,"LINESTRING (80.89637 8.06430, 80.89660 8.06426)",0.0
1,258522956,4086539983,1,0.0,0.0,47.401786,0.022691,trunk,"LINESTRING (80.89778 8.06394, 80.89798 8.06390)",0.0
2,258522957,3213759551,1,0.0,0.0,49.815476,0.061925,trunk,"LINESTRING (80.90017 8.06359, 80.90073 8.06352)",0.0
3,258522958,3213759238,1,0.0,0.0,49.815476,0.031076,trunk,"LINESTRING (80.90121 8.06341, 80.90146 8.06329)",0.0
4,258522959,3213759374,1,0.0,0.0,49.815476,0.051646,trunk,"LINESTRING (80.90204 8.06274, 80.90226 8.06233)",0.0
...,...,...,...,...,...,...,...,...,...,...
9618,7262773170,1147030351,4,0.0,0.0,NaN,0.082320,trunk,"LINESTRING (81.73349 7.67291, 81.73391 7.67230)",0.0
9619,7262813698,1883639938,4,0.0,0.0,NaN,0.002118,trunk,"LINESTRING (81.73567 7.66971, 81.73568 7.66970)",0.0
9620,7294362135,3594153661,3,0.0,0.0,NaN,0.164036,secondary,"LINESTRING (81.03153 8.88533, 81.03266 8.88436)",0.0
9621,7294362140,7294362144,3,0.0,0.0,NaN,0.053804,secondary,"LINESTRING (81.03403 8.88341, 81.03445 8.88316)",0.0


In [52]:
# save as shapefile
#results[0].to_file(driver = 'ESRI Shapefile', filename = "./weighted_sec_saved_edges.shp")

In [53]:
# save as shapefile
#results[1].to_file(driver = 'ESRI Shapefile', filename = "./shortest_path_routes.shp")